In [ ]:
# 生成验证码图片
from captcha.image import ImageCaptcha
# 尺寸[160,60,3]
img = ImageCaptcha()
DIR = "./CaptchaImage/"
def saveImage(text):
    image = img.generate_image(text)
    image.save(DIR+text+'.jpg')
    return
for i in range(10000):
    saveImage(str(i).zfill(4))

In [1]:
import tensorflow as tf
import os
# 进度条的库
from tqdm import tqdm

In [2]:
DIR = "./CaptchaImage/"
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def decodeJPG(sess,DIR,filename):
    image = tf.gfile.FastGFile(os.path.join(DIR,filename),'rb').read()
    # 将图片解码，解成张量  image is a 3-D uint8 or uint16 Tensor of shape [height, width, channels]
    image = tf.image.decode_jpeg(image)
    # 转化为灰度图
    image = sess.run(tf.image.rgb_to_grayscale(image))
    # 将图片转化为1维
    return sess.run(tf.reshape(image,[-1]))

def createTfRecords():
    filename = "./CaptchaRecognize/output.tfrecords"
    with tf.Session() as sess:
        # 创建一个writer来写tfrecord文件
        with tf.python_io.TFRecordWriter(filename) as writer:
            for root,dirs,files in os.walk(DIR):
                total = len(files)
                # 定义进度条
                pbar = tqdm(total = total,desc = '处理进度')
                for file in files:
                    # 将图像转化为字符串
                    image = decodeJPG(sess,DIR,file).tostring()
                    # 获得标签
                    names = file.split('.')
                    label = names[0]
                    # 将一个验证码图片转化为Example Protocol Buffer
                    example = tf.train.Example(features=tf.train.Features(feature={
                        'input_pixels_num':_int64_feature(160*60),
                        'label':_bytes_feature(label.encode()),
                        'image':_bytes_feature(image)
                    }))
                    writer.write(example.SerializeToString())
                    pbar.update(1)
                pbar.close()
createTfRecords()

处理进度:   0%|          | 0/10000 [00:00<?, ?it/s]

Instructions for updating:
Use tf.gfile.GFile.


处理进度:  26%|██▋       | 2626/10000 [3:05:16<17:59:51,  8.79s/it]

ERROR:tensorflow:Session failed to close after 30 seconds. Continuing after this point may leave your program in an undefined state.


KeyboardInterrupt: 